## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-27-02-26-22 +0000,nypost,"NYPD detective, ‘loving’ mom of 3 dies in Colo...",https://nypost.com/2025/10/26/us-news/nypd-det...
1,2025-10-27-02-21-09 +0000,startribune,Takeaways: Timberwolves edge Pacers in home op...,https://www.startribune.com/takeaways-timberwo...
2,2025-10-27-02-18-00 +0000,nyt,What to Know About Argentina’s Midterm Election,https://www.nytimes.com/2025/10/26/world/ameri...
3,2025-10-27-02-18-00 +0000,wsj,"Singapore Dollar, Asian Currencies Strengthen ...",https://www.wsj.com/articles/yen-weakens-amid-...
4,2025-10-27-02-12-00 +0000,wsj,China Industrial Profits Keep Up Strong Growth,https://www.wsj.com/economy/china-industrial-p...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2318/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
50,trump,39
37,mamdani,9
170,melissa,9
48,deal,8
169,hurricane,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
185,2025-10-26-06-49-24 +0000,nypost,"Trump signs peace, trade deals ending conflict...",https://nypost.com/2025/10/26/us-news/trump-si...,77
6,2025-10-27-02-01-40 +0000,bbc,US and China agree framework of trade deal ahe...,https://www.bbc.com/news/articles/c1lqj5lz4geo...,73
130,2025-10-26-13-39-19 +0000,bbc,Will Trump's 'peace deal' solve Thailand-Cambo...,https://www.bbc.com/news/videos/cy40n3ykx93o?a...,72
110,2025-10-26-16-10-00 +0000,wsj,Trump Says He Will Raise Tariffs on Canada by ...,https://www.wsj.com/politics/policy/trump-hike...,68
141,2025-10-26-12-24-18 +0000,nypost,"US, China reach deal to allow TikTok sale, Bes...",https://nypost.com/2025/10/26/us-news/us-china...,68


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
185,77,2025-10-26-06-49-24 +0000,nypost,"Trump signs peace, trade deals ending conflict...",https://nypost.com/2025/10/26/us-news/trump-si...
119,42,2025-10-26-15-13-13 +0000,nypost,Sen. Chris Murphy suggests Dems could end shut...,https://nypost.com/2025/10/26/us-news/sen-chri...
8,40,2025-10-27-01-32-03 +0000,nypost,Hecklers taunt Gov. Hochul with ‘Tax the Rich’...,https://nypost.com/2025/10/26/us-news/hecklers...
165,37,2025-10-26-09-47-45 +0000,nypost,2 Louvre robbery suspects busted in $100M muse...,https://nypost.com/2025/10/26/world-news/louvr...
184,30,2025-10-26-07-02-02 +0000,nypost,Hurricane Melissa strengthens into Category 3 ...,https://nypost.com/2025/10/26/world-news/hurri...
110,29,2025-10-26-16-10-00 +0000,wsj,Trump Says He Will Raise Tariffs on Canada by ...,https://www.wsj.com/politics/policy/trump-hike...
66,29,2025-10-26-20-12-23 +0000,nypost,Gavin Newsom’s story about ‘hustling’ to pay t...,https://nypost.com/2025/10/26/us-news/gavin-ne...
21,27,2025-10-27-00-59-51 +0000,nypost,"US Navy Sea Hawk helicopter, F/A-18F Super Hor...",https://nypost.com/2025/10/26/world-news/us-na...
173,27,2025-10-26-09-00-00 +0000,wsj,Rep. Jared Golden didn’t just side with the GO...,https://www.wsj.com/politics/policy/jared-gold...
145,25,2025-10-26-11-47-44 +0000,nypost,"London police nab sex offender, asylum-seeker ...",https://nypost.com/2025/10/26/world-news/londo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
